In [2]:
"""Needed packages to run everything"""
from pyspark.sql import SparkSession
import yaml

"""General methods that are dictate what stable version the file is currently in, how the file is read, created and how the metadata is dumped into it"""

"""Global source systems used in datasets"""
sources = []

class generalMethods:
    
    def __init__(self, majorVersion: int, minorVersion : int) -> None:
        """
        Constructor of the class general methods 
        ...
        Arguments
        ----------
        self: 
            variable that calls this class to create an instance
        majorVersion : int
            current major version that is stable for instance
        minorVersion : int
            current minor version that is stable for instance
        ...
        Returns
        ----------
        None
        """
        self.majorVersion = majorVersion
        self.minorVersion = minorVersion
        self.stableVersion = f"{self.majorVersion}.{self.minorVersion}"

    def writeMetadataFile(self, schema: dict, outputMetadataPath: str, fileName: str, outputFileType : str = "yaml") -> None:
        """
        Write generated metadata, with the current stable version
        ...
        Arguments
        ----------
        schema : dict
            dictionary that contains all the data that needs to be inputed inside the .yaml file to structure the metadata
        outputMetadataPath : str
            path of where the metadata files are going to be saved, so that they are structured and easy to find
        fileName : str
            path of where each folder is gonna be grouped based on the type of the file
        stableVersion : str
            latest stable version that the metadata will be generated, which is depended on the global variables majorVersion and minorVersion
        outputFileType : str
            output file extension (type) that we want to create, default value is `yaml` if no value is given for that parameter
        ...
        Returns
        ----------
        None
        """
        with open(f'{outputMetadataPath}/{fileName}/{fileName}%{self.stableVersion}.{outputFileType}', 'w') as file:
            yaml.dump(schema, file, sort_keys=False, version=(self.majorVersion, self.minorVersion))

In [3]:

"""Default functions to check if certain values fulfill certain criteria and functions that manipulate with the global "sources" variable"""

"""Mapping string value to boolean so we can use where ever they used"""
strBoolMapping = {
    "y": True,
    "n": False,
    "gdpr": "true"
}

def isNull(nullable : str) -> bool:
    """
    Finding from the dataframe column if that column can have null values (is it required or not)
    ...
    Arguments
    ----------
    nullable : str
        cell value of the row that we are currently checking, on the column "NULLABLE"
    ...
    Returns
    ----------
    Returns a boolean value, based on the variable strBoolMapping
    """
    return strBoolMapping[nullable.lower()]

def isDate(dataType : str) -> bool:
    """
    Function returns extra metadata for column "DATA_TYPE", if that column fulfills a certain condition (is a date type)
    ...
    Arguments
    ----------
    dataType : str
        cell value of the row that we are currently checking, on the column "DATA_TYPE"
    ...
    Returns
    ----------
    Retuns a boolean value, if that specific value has type date
    """
    dateTypeChecked = "date"
    dateBool = dataType.lower() == dateTypeChecked
    return dateBool

def isGdpr(gdprFlag : str) -> bool:
    """
    Function returns extra metadata for column "GDPR_FLAG", if that row fulfills a certain condition (has the cell value "true")
    ...
    Arguments
    ----------
    gdprFlag : str
        cell value of the row that we are currently checking, on the column "GDPR_FLAG"
    ...
    Returns
    ----------
    Boolean value, if the value that we want to compare is equal that to the value that is being compared by 
    """
    gdprBool = gdprFlag.lower() == strBoolMapping["gdpr"]
    return gdprBool

"""Defining the basic functions for reading and specifying the current stable version for the generated metadata file"""
def insertSourceSystem(source : dict) -> None:
    """
    Inserting all the possible source systems into a single global variable
    ...
    Arguments
    ----------
    source : str
        source is the dictionary that a defines the source of a specific table
    ...
    Returns
    ----------
    None
    """
    sources.append(source)

def getSourceSystem(sourceRef: int) -> dict:
    return sources[sourceRef]

def isDistinctSourceSystem(source : str) -> bool:
    """
    Determining if the source system that we are checking has already appeard, or is it the first occurence of this value of source system
    ...
    Arguments
    ----------
    sourceSystem : str
        name of the source system that we are checking
    ...
    Returns 
    ----------
    Boolean value, based on the condition that the source system that we are checking hasn't appeared before
    """
    sourceSystemBool = source not in sources
    return sourceSystemBool

def createSourceSystemRef(fileName : str, decimalSeparator : str, fileType : str) -> int:
    """
    Creating the yaml anchor reference, so that aliases are created faster
    ...
    Arguments
    ----------
    sourceSystem : str
        name of the source system that we want to create reference to
    fileName : str
        name of the file that this table column is part of
    decimalSeparator : str
        value that determines how the decimals are separated
    fileType : str
        the file type that this table column is part of
    ...
    Returns
    ----------
    Index of element that corresponds with source in the list of sources
    """
    source = {
        "load_type": "delta",
        "file_pattern": fileName,
        "params": {
            "decimal_separator": decimalSeparator,
            "format": fileType.lower()
        }
    }
    if isDistinctSourceSystem(source):
        insertSourceSystem(source)
    return sources.index(source)


In [4]:
"""Needed packages to run everything in this file"""
from pyspark.sql import DataFrame

"""Functions that generate all the metadata for different parts of a dataframe """
    
def genPrimaryArgsMeta(title : str, sourceSystem : str, description : str = "A metaschema generated from an example dataset.") -> dict:
    """
    Write general info of metadata file
    ...
    Arguments
    ----------
    title : str
        variable that is gonna be used for the title of the metadata file
    sourceSystem : 
    description : str
        variable that is gonna be used for the description of the metadata file, also has a default value which is gonna be used when there is not value give
    ...
    Returns
    ----------
    It returns a dictionary of the values mentioned above which are gonna be used later to build the entires metadata file
    """
    version = 2
    primaryArgSchema = {
        "version": version,
        "source_system": sourceSystem,
        "title": title,
        "description": f"{description}",
        "tables": []
    }
    return primaryArgSchema
    
def genTableArgsMeta(tableName : str, sourceRef : int, inputSchema : list[dict]) -> dict:
    """
    Generating primary attributes for metadata schema
    ...
    Arguments
    ----------
    tableName : str
        name of the table that we are generating metadata for
    sourceSystem : str
        name of the source system that the table column is part of
    inputSchema : list[dict]
        list of dictionaries that are placed in the `columns` attribute
    ...
    Returns
    ----------
    A dictionary of the primary metadata attributes for the table that we are checking 
    """
    tableArgMeta = {
        "name": f"{tableName}",
        "source": getSourceSystem(sourceRef),
        "columns": inputSchema,
        "primary_key": "unid",
        "cdc_column": "dml_flag",
        "cdc_type": "soft" 
    }
    return tableArgMeta

def genTableArgsGdpr(tableName : str, inputSchema : list[dict]) -> dict:
    """
    Generating primary attributes for gdpr schema
    ...
    Arguments
    ----------
    tableName : str
        name of the table that we are generating metadata for
    inputSchema : list[dict]
        list of dictionaries that are placed in the `personal < hash` attribute
    ...
    Returns
    ----------
    A dictionary of the primary gdpr attributes for the table that we are checking 
    """
    tableArgGdpr = {
        "name": tableName,
        "personal_data": {"hash" : inputSchema}
    }
    return tableArgGdpr

def genColumnSchema(tableDataframe : DataFrame) -> dict:
    """
    Generating the schema metadata for each column on a table
    ...
    Arguments
    ----------
    tableDataframe : DataFrame
        a piece of the original dataframe that has the column "table_name" equal to a specific table name that enables us to see all the columns in that specific piece of dataframe
    ...
    Returns
    ----------
    Function returns a dictionary that contains metadata for each dataframe column (like: name, data_type, is_nullable, date_format etc)
    """
    columnSchema = {
        "columns": [],
        "hash": []
    }
    for row in tableDataframe:
        columnName = row.COLUMN_NAME.lower()
        dataType = row.DATA_TYPE.lower()
        if "number" in dataType:
            dataType = dataType.replace("number", "decimal")
        elif "varchar" in dataType:
            dataType = "string"
        column_info = {
            "name": columnName,
            "data_type": dataType
        }
        if isNull(row.NULLABLE):
            column_info["is_nullable"] = True
        if isDate(row.DATA_TYPE):
            column_info["date_format"] = "yyyy-MM-dd HH:mm:ss"
        if isGdpr(row.GDPR_FLAG):
            columnSchema["hash"].append(columnName)
        columnSchema["columns"].append(column_info)
    return columnSchema

In [5]:
"""Needed packages to run everything"""
from pyspark.sql import DataFrame

"""All the functions to generate the schemas are called here and that data is saved in the variable "schema\""""

def datasetTableInfo(dataFrame : DataFrame, metadataTitle : str, description : str = "A metaschema generated from an example dataset.") -> dict:
    """
    All the pieces of the whole schema are generated here and are merged together
    ...
    Arguments
    ----------
    dataFrame : DataFrame
        the dataframe that we are gonna generate all the schema for
    metadataTitle : str
        title of the schema file that we are gonna call it
    description : str
        description about the schema, if this argument is not sent the default value is used
    ...
    Returns
    ----------
    The whole generated schema about this dataframe
    """
    sourceSystem = dataFrame.collect()[0].SOURCE_SYSTEM
    completeSchema = {
        "metadata": genPrimaryArgsMeta(metadataTitle, sourceSystem, description),
        "gdpr": []
    }
    distinctTable_name = dataFrame.dropDuplicates(["table_name"]).collect()
    for row in distinctTable_name:
        tableName = row.table_name
        singleTableDataframe = dataFrame.filter(dataFrame.table_name == tableName).collect()
        sourceRef = createSourceSystemRef(row.FILE_NAME, row.DECIMAL_SEPARATOR, row.FILE_TYPE)
        columnSchema = genColumnSchema(singleTableDataframe)
        completeSchema["metadata"]["tables"].append(genTableArgsMeta(tableName, sourceRef, columnSchema["columns"]))
        completeSchema["gdpr"].append(genTableArgsGdpr(tableName, columnSchema["hash"]))
    return completeSchema
        
    

In [7]:
import time

"""Basic variables that define current stable version of schemas, "SparkSession" object, paths of where to read and save files and calling functions that generate and dump the schema to those files"""

"""Gloabl varibles for major and minor stable metadata and gdpr versions"""
majorMetadataVersion = 1
minorMetadataVersion = 24
metadata = generalMethods(majorMetadataVersion, minorMetadataVersion)

majorGdprVersion = 1
minorGdprVersion = 9
stableGdprVersion = generalMethods(majorGdprVersion, minorGdprVersion)

"""Object of SparkSession that is gonna be used for all the methods of pySpark"""
spark = SparkSession.builder.getOrCreate()

"""Global input metadata path"""
inputDataFramePath = "../Input CSV Metadata/20240101_metadata"

"""Global output metadata path"""
outputSchemaPath = "../Output Metadata/Schema Versions"
metadataFileName = "Metadata"
gdprFileName = "Gdpr"

"""Saving the dataframe into a variable so that we don't have to call the function readCSV() each time!"""
dataFrame = spark.read.csv(f"{inputDataFramePath}.csv", header=True, sep=';').cache()

"""Start the timer, not including the time it takes to read the csv"""
beginTime = time.time()

"""Generating the metadata from the dataset and saving it into a variable"""
primaryArgSchema = datasetTableInfo(dataFrame, "TestMetadata")
metadataSchema = primaryArgSchema["metadata"]
gdprSchema = primaryArgSchema["gdpr"]

"""Using generated metadata, output paths and the current latest stable version to dump the metadata into a .yaml schema """
metadata.writeMetadataFile(metadataSchema, outputSchemaPath, metadataFileName)
metadata.writeMetadataFile(gdprSchema, outputSchemaPath, gdprFileName)

"""Stop the spark session"""
spark.stop()
